In [30]:
import os
import sys
import numpy as np
print(os.getcwd())
print(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.getcwd()))
import pandas as pd
import epitran
from datasets import load_dataset, DatasetDict
from functools import lru_cache
from sklearn.model_selection import train_test_split

/home/toure215/BERT_phonetic/test
/home/toure215/BERT_phonetic


In [2]:
dataset = load_dataset("csv" , data_files="/home/toure215/BERT_phonetic/DATASETS/verses.csv")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Verse', 'Meter', 'char_count'],
        num_rows: 199002
    })
})

In [4]:
pd_dataset = pd.read_csv("/home/toure215/BERT_phonetic/DATASETS/verses.csv")
pd_dataset.rename(columns={"Verse":"text"}, inplace=True)
pd_dataset.head()

,id,text,Meter,char_count
0,0,ah why this boding start this sudden pain,iambic,6
1,1,that wings my pulse and shoots from vein to vein,iambic,6
2,2,what mean regardless of yon midnight bell,iambic,6
3,3,these earthborn visions saddening o'er my cell,iambic,6
4,4,what strange disorder prompts these thoughts t...,iambic,6


In [5]:
pd_dataset.iloc[1]["text"]

'that wings my pulse and shoots from vein to vein'

In [6]:
verse_dataset = pd.DataFrame(columns=["id", "sentence1", "sentence2", "label"])
verse_dataset

,id,sentence1,sentence2,label


In [7]:
epi = epitran.Epitran("eng-Latn")
@lru_cache(maxsize=None)
def cached_xsampa_list(word):
    return epi.xsampa_list(word)

def is_rhyming(word1, word2):
    sound1 = cached_xsampa_list(word1)
    sound2 = cached_xsampa_list(word2)
    if len(sound1) < 2 or len(sound2) < 2:
        return False
    return sound1[-2:] == sound2[-2:]

In [8]:
for i in range(0, len(pd_dataset) - 1, 2):
    last = len(verse_dataset)
    verse_dataset.loc[last] = [
        last,
        pd_dataset.iloc[i]["text"],
        pd_dataset.iloc[i + 1]["text"],
        1, # rhyming
    ]

    max_attempts = 10
    attempts = 0
    while attempts < max_attempts:
        random_index1 = np.random.randint(0, len(pd_dataset))
        random_index2 = np.random.randint(0, len(pd_dataset))
        word1 = pd_dataset.iloc[random_index1]["text"].split()[-1]
        word2 = pd_dataset.iloc[random_index2]["text"].split()[-1]
        if not is_rhyming(word1, word2):
            break
        attempts += 1

    verse_dataset.loc[last + 1] = [
        last + 1,
        pd_dataset.iloc[random_index1]["text"],
        pd_dataset.iloc[random_index2]["text"],
        0,  # not rhyming
    ]

In [9]:
print(len(verse_dataset))
print(len(pd_dataset))
verse_dataset.head(20)

199002
199002


,id,sentence1,sentence2,label
0,0,ah why this boding start this sudden pain,that wings my pulse and shoots from vein to vein,1
1,1,the best kind blessings heaven can grant the fair,clos'd in a proud seraglio's gloomy walls,0
2,2,what mean regardless of yon midnight bell,these earthborn visions saddening o'er my cell,1
3,3,farewel reply'd his friend may you advance,careless of all th' anxiety he feels,0
4,4,what strange disorder prompts these thoughts t...,these sighs to murmur and these tears to flow,1
5,5,and plausible than social life requires,resistless in so bad a cause but lame,0
6,6,'tis she 'tis eloisa's form restor'd,once a pure saint and more than saints ador'd,1
7,7,well may your hearts believe the truths i tell,broods she in purple o'er the venal bar,0
8,8,she comes in all her killing charms confest,glares thro' the gloom and pours upon my breast,1
9,9,and let my fate your pity move,prison my heart in thy steel bosoms ward,0


In [11]:
print("Number of unique rows:", verse_dataset[["sentence1", "sentence2", "label"]].drop_duplicates().shape[0])

Number of unique rows: 199002


In [18]:
verse_dataset.to_csv("/home/toure215/BERT_phonetic/DATASETS/verse_dataset.csv", index=False)

In [19]:
train, test = train_test_split(verse_dataset, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.1, random_state=42)

In [20]:
train.to_csv("/home/toure215/BERT_phonetic/DATASETS/verses/train.csv", index=False)
val.to_csv("/home/toure215/BERT_phonetic/DATASETS/verses/val.csv", index=False)
test.to_csv("/home/toure215/BERT_phonetic/DATASETS/verses/test.csv", index=False)

In [28]:
train = load_dataset("csv", data_files="/home/toure215/BERT_phonetic/DATASETS/verses/train.csv")
val = load_dataset("csv", data_files="/home/toure215/BERT_phonetic/DATASETS/verses/val.csv")
test = load_dataset("csv", data_files="/home/toure215/BERT_phonetic/DATASETS/verses/test.csv")

In [29]:
print(train)
print(val)
print(test)

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 143280
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 15921
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 39801
    })
})


In [32]:
hf_dataset = DatasetDict({
    "train": train["train"],
    "validation": val["train"],
    "test": test["train"],
})
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 143280
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 15921
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 39801
    })
})

In [33]:
hf_dataset.save_to_disk("/home/toure215/BERT_phonetic/DATASETS/verses/hf_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/143280 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15921 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/39801 [00:00<?, ? examples/s]